# Load Libs

In [17]:
### Load Python Lib##### 
########
import numpy as np
from math import *
import matplotlib.pyplot as plt
from random import gauss, randint
from matplotlib import rc
from matplotlib.ticker import FixedLocator, MultipleLocator, FormatStrFormatter
import time
from scipy.special import spence
import scipy.optimize
from scipy import stats
import itertools
import iminuit
#import probfit

#very import to enable this so the plots can be showed in the page
get_ipython().magic(u'matplotlib inline')

# Use LaTeX font.
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Computer Modern Roman'],'size':20})

import matplotlib.font_manager as font_manager
font_prop = font_manager.FontProperties(size=12)

Deg2Rad = 3.1415926/180.0

# Load MC Data

In [18]:
### Loading the theoretical calculations
filename1 = 'database/n_asym_0_fine12.dat'
lines1 = open(filename1,'r').readlines()

N = 7
W = np.zeros(N, dtype=float)
Q2 = np.zeros(N, dtype=float)
x = np.zeros(N, dtype=float)
t = np.zeros(N, dtype=float)
Epsilon = np.zeros(N, dtype=float)
R = np.zeros(N, dtype=float)
F = np.zeros(N, dtype=float)     
XS = np.zeros(N, dtype=float)  
Asym= np.zeros(N, dtype=float)  
Astat = np.zeros(N, dtype=float) 
N_out = np.zeros(N, dtype=float)   
N_raw = np.zeros(N, dtype=float)   
N_pim = np.zeros(N, dtype=float)   

N1 = 12
N2 = 12
Bin = (N, N1, N2)
PhiS = np.zeros(Bin, dtype=float)
PhiH = np.zeros(Bin, dtype=float)
Ncnt = np.zeros(Bin, dtype=float)
Nstat = np.zeros(Bin, dtype=float)

#BinS = np.zeros((N, N1), dtype=int)
#BinH = np.zeros((N, N2), dtype=int)

## Reading in each -t bin
N_Bin =0
Q2_avg = 0.0
W_avg = 0.0
for i in range(0, N):
    values=lines1[N_Bin].split()
    values = np.array(values,dtype=float)
    Q2[i]=(values[1])
    x[i]=(values[2])
    W[i]=(values[3])
    t[i]=(values[4])
    Epsilon[i]=(values[5])
    R[i]=(values[6])
    F[i]=(values[7])
    XS[i]=(values[8])
    Asym[i]=(values[9])
    Astat[i]=(values[10])
    N_out[i]=(values[11])
    N_raw[i]=(values[12])
    
    Q2_avg += Q2[i]
    W_avg += W[i]
    
    ## Reading in Phi_S and Phi_h bins for each -t bin
    for j in range(0, N1):
        for k in range(0, N2):
            N_phi = (N_Bin+1) + k + j*N2
            values=lines1[N_phi].split()
            values = np.array(values, dtype=float)
                       
            if j!=int(values[0]) or k!=int(values[1]):
               print "*** ERROR, j!=%d/%d, k!=%d/%d"%(j, int(values[0]), k, int(values[1]))
            
            PhiS[i][j][k] = values[2] * Deg2Rad
            PhiH[i][j][k] = values[3] * Deg2Rad
            Ncnt[i][j][k] = values[4]
            Nstat[i][j][k] = values[5]
            #print "i=%d: %d  %d  %12.4f  %12.4f  %12.4e  %12.4e"%(i,j,k, PhiS[i][j][k]/Deg2Rad, PhiH[i][j][k]/Deg2Rad, Ncnt[i][j][k], Nstat[i][j][k])  
    
    N_Bin+=N1*N2 + 1
    
Q2_avg /= N
W_avg /=N

# Asymmetry Fitting

## Load Data from one -t bin

In [19]:
## Load only one bin
def LoadData(iii):
    phi_S1 = np.zeros(N1*N2, dtype=float)
    phi_H1 = np.zeros(N1*N2, dtype=float)
    F_UT1 = np.zeros(N1*N2, dtype=float)
    dF_UT1 = np.zeros(N1*N2, dtype=float)
    #Ncnt1 = np.zeros(N1*N2, dtype=int)

    #F_UU1 = np.zeros(N1*N2, dtype=float)
    #F_UT_TEST1 = np.zeros(N1*N2, dtype=float)
    
    phi1 = np.zeros((N1*N2,2), dtype=float)
    i=0
    for j in range(0, N1):
        for k in range(0,N2):
            phi_S1[i] = PhiS[iii][j][k]
            phi_H1[i] = PhiH[iii][j][k]
            
            phi1[i] = [phi_S1[i], phi_H1[i]]
            
            #F_UT[i] = Ncnt[iii][j][k]
            if Nstat[iii][j][k]<1.0:
                F_UT1[i] = 0.0
                dF_UT1[i] = 0.0
            else:
                F_UT1[i] = (Ncnt[iii][j][k])/Nstat[iii][j][k]
            if Ncnt[iii][j][k]<1.0:
                dF_UT1[i] = 0.0
            else:
                dF_UT1[i] = 1./sqrt(Ncnt[iii][j][k])
            
            #Ncnt[i] = Ncnt[iii][j][k]

            #F_UU1[i] = Nstat[iii][j][k]
            #F_UT_TEST1[i] = 1.0 + Asym[iii] * sin(3.0*phi_S1[j][k]+phi_H1[j][k])
            #if j==2 and k==2:
            #    print "A_UT = %f / %f,  R = %f"%(F_UT[j][k],F_UT_TEST[j][k],F_UT[j][k]/F_UT_TEST[j][k])
            
            i+=1
            
    return phi_S1, phi_H1, phi1, F_UT1, dF_UT1

## Using Maximum-Likelihood Fit

### Fitting Function 1:

In [20]:
def asym(phi, a, b):
        phi1 = phi[0]
        phi2 = phi[1]
        return a + b*sin(3.0*phi1 + phi2)

def func(phi_s, phi_h, a, b):
        return a + b*sin(3.0*phi_s + phi_h)

In [21]:
def chi2func(a,b):
    chi2 = 0.0
    for i in range(0, len(phi_S)):
        model = func(phi_S[i], phi_H[i], a, b)
        err = (model - F_UT[i])/dF_UT[i]
        chi2 += err**2
    
    return chi2
    

In [74]:
def likfunc(a,b):
    L = 0.0
    for i in range(0, len(phi_S)):
        
        #phi_S_Bin = np.random.uniform(phi_S[i]-15.0, phi_S[i]+15.0, N[i])
        #phi_H_Bin = np.random.uniform(phi_H[i]-15.0, phi_H[i]+15.0, N[i])
        model = func(phi_S_Bin[i], phi_H_Bin[i], a, b)
        #model = np.array([func(phi_S_Bin[j], phi_H_Bin[j], a, b) for j in range(N[i])])
        pdf = model**N[i]*np.exp(-model)/np.prod
        L += (np.log(pdf))
    
    return L

### Do the TMinuit Fit:

In [75]:
I=3
phi_S, phi_H, phi_SH, F_UT, dF_UT = LoadData(I)
N = (1./(dF_UT**2))
N = np.array(N, dtype=int)

In [76]:
## Do the minimization
print '-- Input: a = 1.0, b = %f+/- %f'%(Asym[I], Astat[I])
minuit = iminuit.Minuit(likfunc, a=1.0, b=Asym[I], error_a = 1e-3, error_b=1e-3) # see iminuit tutorial on how to give initial value/range/error
minuit.errordef=0.5
minuit.migrad() # MIGRAD is a very stable robust minimization method
#minuit.hesse()
#minuit.print_param()
minuit.print_matrix()
results = minuit.values
errors = minuit.errors
print "A = %f +/- %f, Fit = %f +/-%f"%(Asym[I], Astat[I], results['b'], errors['b'])

-- Input: a = 1.0, b = -0.236200+/- 0.007060


/Users/yez/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:3: InitialParamWarning: errordef is not given. Default to 1.
  app.launch_new_instance()
/Users/yez/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in log
/Users/yez/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:5: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
    a = -7236209.623544
    b = +102104772.513936

/Users/yez/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:5: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
    a = nan
    b = nan

/Users/yez/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:5: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
    a = +8.428711
    b = -663654627.796356



FCN = -844.303603869 
 TOTAL NCALL = 65 
 NCALLS = 65 
 
 
 EDM = 2.67876188456e+20 
 GOAL EDM = 5e-06 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 False 
 False 
 True 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,a,1,3.21267e-10,0,0,,,
2,b,-0.2362,0.0286721,0,0,,,


+,a,b
a,1.00,-1.00
b,-1.00,1.00


A = -0.236200 +/- 0.007060, Fit = -0.236200 +/-0.028672
